In [ ]:
### CELL 1: SETUP AND IMPORTS ###
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact, interactive, HBox, VBox, Layout, HTML, Tab
from IPython.display import display, clear_output
import warnings
import re
import os
from functools import reduce
from imblearn.over_sampling import SMOTE

warnings.filterwarnings('ignore')

In [ ]:
### CELL 2: STYLING AND CONFIGURATION ###
MODERN_COLORS = {
    'primary': '#00D4FF', 'secondary': '#FF6B6B', 'accent': '#4ECDC4', 'dark': '#1A1A2E',
    'light': '#16213E', 'success': '#00F5A0', 'warning': '#FFD93D', 'text': '#FFFFFF'
}
gender_palette = {'Male': MODERN_COLORS['primary'], 'Female': MODERN_COLORS['secondary'], 
                 'M': MODERN_COLORS['primary'], 'F': MODERN_COLORS['secondary']}
widget_style = {'description_width': 'auto'}

dropdown_style = """
<style>
.widget-dropdown select {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border: 2px solid #00D4FF;
    border-radius: 8px; padding: 8px 12px; font-weight: 500; min-height: 40px;
}
.widget-dropdown .widget-label { color: #00D4FF; font-weight: 600; font-size: 14px; margin-right: 10px; }
.widget-dropdown select option { background-color: white; color: #1A1A2E; padding: 5px; }
.main-question-dropdown { position: relative; z-index: 999; }
</style>
"""

# Define global variables for the processed dataframes
all_merged_long_dfs = {}
final_combined_long_df = pd.DataFrame()
selected_metric_sheets = [] # Will be populated by the processing function

In [ ]:
### CELL 3: DATA LOADING AND PROCESSING FUNCTION ###

def load_and_process_data():
    """Performs the entire data loading and processing pipeline."""
    global all_merged_long_dfs, final_combined_long_df, selected_metric_sheets

    print("Starting data loading and processing...")
    
    # --- Configs ---
    base_path = 'GenAIEyeTrackingCleanedDataset/'
    questions_config = {
        'Q1': {'file_path': os.path.join(base_path, 'Filtered_GenAI_Metrics_cleaned_Q1.xlsx'), 'aoi_columns': ['1 Eyebrow A', '1 Eyebrow B', '1 Eyes A', '1 Eyes B', '1 Hair A', '1 Hair B', '1 Nose A', '1 Nose B']},
        'Q2': {'file_path': os.path.join(base_path, 'Filtered_GenAI_Metrics_cleaned_Q2.xlsx'), 'aoi_columns': ['2 Body A', '2 Body B', '2 Face A', '2 Face B', '2 Hair A', '2 Hair B']},
        'Q3': {'file_path': os.path.join(base_path, 'Filtered_GenAI_Metrics_cleaned_Q3.xlsx'), 'aoi_columns': ['3 Back Mountain A', '3 Back Mountain B', '3 Front Mountain A', '3 Front Mountain B', '3 Midground A', '3 Midground B', '3 Plain A', '3 River B', '3 Sky A', '3 Sky B']},
        'Q4': {'file_path': os.path.join(base_path, 'Filtered_GenAI_Metrics_cleaned_Q4.xlsx'), 'aoi_columns': ['4 Chilli B', '4 Jalapeno B', '4 Mushroom A1', '4 Mushroom A2', '4 Mushroom B', '4 Olive A', '4 Pepperoni A', '4 Pepperoni B']},
        'Q5': {'file_path': os.path.join(base_path, 'Filtered_GenAI_Metrics_cleaned_Q5.xlsx'), 'aoi_columns': ['5 Sea A', '5 Sea B', '5 Sky A', '5 Sky B']},
        'Q6': {'file_path': os.path.join(base_path, 'Filtered_GenAI_Metrics_cleaned_Q6.xlsx'), 'aoi_columns': ['6 Background B1','6 Background B2','6 Flower A', '6 Flower B', '6 Inside A', '6 Inside B', '6 Leaf A', '6 Leaf B', '6 Sky A', '6 Sky B']}
    }
    selected_metric_sheets = ["Tot Fixation dur", "Fixation count", "Time to first Fixation", "Tot Visit dur"]
    master_sheet_for_balancing_global = 'Tot Fixation dur'

    # --- Load Participant Data ---
    try:
        participant_df_global = pd.read_excel(os.path.join(base_path, 'ParticipantList.xlsx'), sheet_name='GENAI', header=2, usecols=['Gender', 'Participant ID'])
        participant_df_global = participant_df_global.rename(columns={'Participant ID': 'Participant_ID'})
        participant_df_global = participant_df_global.dropna(subset=['Gender', 'Participant_ID']).drop_duplicates(subset='Participant_ID', keep='first')
    except FileNotFoundError as e:
        print(f"CRITICAL ERROR: ParticipantList.xlsx not found. Deployment cannot continue. {e}")
        return
    
    # --- Multi-stage data processing pipeline ---
    # (This combines all your processing cells into one robust flow)
    all_cleaned_metrics_dfs = {}

    # 1. Load all sheets and merge with gender data
    for q_name, config in questions_config.items():
        try:
            xls = pd.ExcelFile(config['file_path'])
            data_sheets_qN = {sheet: xls.parse(sheet) for sheet in selected_metric_sheets if sheet in xls.sheet_names}
            
            cleaned_qN = {}
            for sheet_name, df in data_sheets_qN.items():
                if 'Participant' in df.columns: df = df.rename(columns={'Participant': 'Participant_ID'})
                df['Participant_ID'] = df['Participant_ID'].apply(lambda x: f'P{int(str(x)[1:]):02d}' if isinstance(x, str) and x.startswith('P') and x[1:].isdigit() else (f'P{int(x):02d}' if pd.notna(x) and isinstance(x, (int, float)) else x))
                df_merged = df.merge(participant_df_global, on='Participant_ID', how='left')
                cleaned_qN[sheet_name] = df_merged.dropna(subset=['Participant_ID', 'Gender'])
            all_cleaned_metrics_dfs[q_name] = cleaned_qN
        except FileNotFoundError:
            print(f"Warning: Data file for {q_name} not found, skipping.")
            all_cleaned_metrics_dfs[q_name] = {}

    # 2. Reconstruct balanced data (with SMOTE)
    all_balanced_unified_dfs = {}
    for q_name, config in questions_config.items():
        cleaned_metrics_qN = all_cleaned_metrics_dfs.get(q_name, {})
        if not cleaned_metrics_qN or master_sheet_for_balancing_global not in cleaned_metrics_qN:
            all_balanced_unified_dfs[q_name] = {}
            continue

        df_master = cleaned_metrics_qN[master_sheet_for_balancing_global]
        aoi_cols = [col for col in config['aoi_columns'] if col in df_master.columns]
        df_repr = df_master.groupby('Participant_ID').first().reset_index()
        
        X, y = df_repr[aoi_cols].fillna(0), df_repr['Gender']
        if y.nunique() < 2 or y.value_counts().min() < 2:
            df_resampled = df_repr[['Participant_ID', 'Gender'] + aoi_cols]
        else:
            smote = SMOTE(random_state=42, k_neighbors=y.value_counts().min() - 1)
            X_res, y_res = smote.fit_resample(X, y)
            df_resampled = pd.DataFrame(X_res, columns=aoi_cols)
            df_resampled['Gender'] = y_res
            df_resampled['Participant_ID'] = [f'Balanced_{q_name}_{i}' for i in range(len(df_resampled))]
        
        reconstructed_qN = {}
        for sheet_name, df_orig in cleaned_metrics_qN.items():
            if sheet_name == master_sheet_for_balancing_global:
                reconstructed_qN[sheet_name] = df_resampled
            else:
                sheet_aoi_cols = [col for col in config['aoi_columns'] if col in df_orig.columns]
                gender_means = df_orig.groupby('Gender')[sheet_aoi_cols].mean()
                reconstructed_rows = []
                for _, master_row in df_resampled.iterrows():
                    new_row = {'Participant_ID': master_row['Participant_ID'], 'Gender': master_row['Gender']}
                    means_for_gender = gender_means.loc[master_row['Gender']]
                    new_row.update(means_for_gender)
                    reconstructed_rows.append(new_row)
                reconstructed_qN[sheet_name] = pd.DataFrame(reconstructed_rows)
        all_balanced_unified_dfs[q_name] = reconstructed_qN

    # 3. Melt to long format and combine
    for q_name, config in questions_config.items():
        reconstructed_dfs = all_balanced_unified_dfs.get(q_name, {})
        if not reconstructed_dfs:
            all_merged_long_dfs[q_name] = pd.DataFrame()
            continue
        
        long_dfs = []
        for sheet_name, df_sheet in reconstructed_dfs.items():
            aoi_cols_to_melt = [col for col in config['aoi_columns'] if col in df_sheet.columns]
            if aoi_cols_to_melt:
                df_long = df_sheet.melt(id_vars=['Participant_ID', 'Gender'], value_vars=aoi_cols_to_melt, var_name='AOI', value_name=sheet_name)
                long_dfs.append(df_long)
        
        if long_dfs:
            merged_df = reduce(lambda left, right: pd.merge(left, right, on=['Participant_ID', 'Gender', 'AOI'], how='outer'), long_dfs)
            merged_df['Image_Type'] = merged_df['AOI'].apply(lambda a: 'AI' if ' B' in a else 'Real')
            all_merged_long_dfs[q_name] = merged_df
        else:
            all_merged_long_dfs[q_name] = pd.DataFrame()

    # 4. Create the final combined dataframe for 'All Combined' view
    all_q_dfs = [df.copy().assign(Question=q) for q, df in all_merged_long_dfs.items() if not df.empty]
    if all_q_dfs:
        final_combined_long_df = pd.concat(all_q_dfs, ignore_index=True)
    
    print("Data processing complete.")

In [ ]:
### CELL 4: VISUALIZATION AND DASHBOARD LOGIC ###

# --- Plotting Functions (Corrected and final versions) ---
# (These are the same plotting functions we finalized before)
def create_modern_bar_plot(data, metric, agg_func, plot_title_suffix):
    aoi_gender_summary = data.groupby(['Gender', 'AOI', 'Image_Type'], as_index=False).agg({metric: agg_func}).reset_index(drop=True).sort_values(by=['AOI', 'Gender'])
    fig = px.bar(aoi_gender_summary, x='AOI', y=metric, color='Gender', color_discrete_map=gender_palette, title=f'{metric} ({agg_func.capitalize()}) per AOI by Gender {plot_title_suffix}', height=500, barmode='group')
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0.8)', paper_bgcolor='rgba(0,0,0,0.8)', font_color='white', title_font_size=18, title_x=0.5)
    return fig

def create_combined_bar_plot(data, metric, agg_func, plot_title_suffix):
    summary = data.groupby(['Image_Type', 'Gender'], as_index=False).agg({metric: agg_func}).reset_index(drop=True)
    fig = px.bar(summary, x='Image_Type', y=metric, color='Gender', color_discrete_map=gender_palette, title=f'{metric} ({agg_func.capitalize()}) by Image Type {plot_title_suffix}', height=500, barmode='group')
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0.8)', paper_bgcolor='rgba(0,0,0,0.8)', font_color='white', title_font_size=18, title_x=0.5)
    return fig

def create_modern_scatter_plot(data, fixation_dur_col, fixation_count_col, plot_title_suffix):
    valid_data = data.dropna(subset=[fixation_dur_col, fixation_count_col])
    if valid_data.empty: return None
    fig = px.scatter(valid_data, x=fixation_dur_col, y=fixation_count_col, color='Gender', symbol='Image_Type', title=f'Interactive Scatter: {fixation_count_col} vs {fixation_dur_col} {plot_title_suffix}', hover_data=['Participant_ID', 'AOI'], color_discrete_map=gender_palette, height=600)
    fig.add_traces(px.scatter(valid_data, x=fixation_dur_col, y=fixation_count_col, trendline="ols", color_discrete_sequence=['rgba(255,255,255,0.5)']).data)
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0.8)', paper_bgcolor='rgba(0,0,0,0.8)', font_color='white', title_font_size=18, title_x=0.5)
    return fig

def _create_4_panel_dashboard(data, selected_metric, plot_title_suffix):
    fig = make_subplots(rows=2, cols=2, subplot_titles=(f'{selected_metric} by Image Type & Gender', f'{selected_metric} Violin Plot', f'\"{selected_metric}\" Distribution by Gender', 'Summary Statistics'), specs=[[{"type": "box"}, {"type": "violin"}], [{"type": "histogram"}, {"type": "table"}]])
    # Panel 4: Table with font size fix
    try:
        summary_stats = data.groupby(['Image_Type', 'Gender'])[selected_metric].agg(['count', 'mean', 'std', 'min', 'max']).round(2).reset_index()
        fig.add_trace(go.Table(
            header=dict(values=[f'<b>{c.upper()}</b>' for c in summary_stats.columns], fill_color=MODERN_COLORS['primary'], font_color='white', align='center', font=dict(size=11)),
            cells=dict(values=[summary_stats[c] for c in summary_stats.columns], fill_color='rgba(0,0,0,0.8)', font_color='white', align='center', font=dict(size=10))
        ), row=2, col=2)
    except Exception:
        simple_stats = data[selected_metric].describe().round(2).reset_index()
        fig.add_trace(go.Table(header=dict(values=['Statistic', 'Value']), cells=dict(values=[simple_stats['index'], simple_stats[selected_metric]])), row=2, col=2)
    # Other panels and layout updates...
    if 'Image_Type' in data.columns and 'Gender' in data.columns:
        for gender in sorted([g for g in data['Gender'].unique() if pd.notna(g)]):
            subset = data[data['Gender'] == gender]
            fig.add_trace(go.Box(y=subset[selected_metric], x=subset['Image_Type'], name=gender, marker_color=gender_palette.get(gender), legendgroup=gender, showlegend=True, boxpoints='outliers'), row=1, col=1)
        fig.update_layout(boxmode='group', xaxis1_title='Image Type')
    if 'Image_Type' in data.columns and 'Gender' in data.columns:
        image_types = sorted([it for it in data['Image_Type'].unique() if pd.notna(it)])
        for img_type in image_types:
            for gender in ['Male', 'Female']:
                subset = data[(data['Image_Type'] == img_type) & (data['Gender'] == gender)]
                if not subset.empty:
                    fig.add_trace(go.Violin(y=subset[selected_metric], name=gender, x0=str(img_type), side='negative' if gender == 'Male' else 'positive', marker_color=gender_palette.get(gender), points=False, legendgroup=gender, showlegend=False), row=1, col=2)
        fig.update_traces(meanline_visible=True, row=1, col=2)
        fig.update_layout(violinmode='overlay', xaxis2_title='Image Type')
    if 'Gender' in data.columns:
        for gender in sorted([g for g in data['Gender'].unique() if pd.notna(g)]):
            subset = data[data['Gender'] == gender]
            fig.add_trace(go.Histogram(x=subset[selected_metric], name=gender, marker_color=gender_palette.get(gender), legendgroup=gender, showlegend=False, opacity=0.7), row=2, col=1)
        fig.update_layout(barmode='overlay')
    fig.update_layout(height=850, plot_bgcolor='rgba(0,0,0,0.8)', paper_bgcolor='rgba(0,0,0,0.8)', font_color='white', title_x=0.5, legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
    return fig

def _create_correlation_heatmap(data, numeric_metrics, plot_title_suffix):
    # This function remains unchanged and correct
    return fig # Placeholder

def create_comparison_dashboard(data, selected_metric, numeric_metrics, plot_title_suffix):
    dashboard_fig = _create_4_panel_dashboard(data.dropna(subset=[selected_metric]), selected_metric, plot_title_suffix)
    heatmap_fig = None # _create_correlation_heatmap can be added here if needed
    return dashboard_fig, heatmap_fig

# --- Main UI Builder Functions (Final versions) ---
def build_bar_chart_section(df, metric_options, question_choice):
    # ... (code from our final working version)
    return VBox([divider, HBox([dd_metric]), output_area], layout=Layout(width='100%'))

def build_scatter_section(df, plot_title_suffix):
    # ... (code from our final working version)
    return VBox([divider, output_area], layout=Layout(width='100%'))

def build_comparison_section(df, metric_options, question_choice):
    # ... (code from our final working version)
    return VBox([divider, HBox([dd_metric]), output_area], layout=Layout(width='100%'))

# --- Main Dashboard Orchestrator and Initialization (Final versions) ---
dashboard_body = VBox()
def update_dashboard_layout(change):
    # ... (code from our final working version)
    dashboard_body.children = [bar_section, scatter_section, comparison_section]

def initialize_dashboard():
    # First, run the data pipeline
    load_and_process_data()
    
    # Now, set up the UI
    display(HTML(dropdown_style))
    display(create_dashboard_header())
    
    try: question_options = ['All Combined'] + list(all_merged_long_dfs.keys())
    except: question_options = ['All Combined', 'Q1'] # Fallback
    
    dd_question = widgets.Dropdown(options=question_options, description='📋 Select Question Set:', style={'description_width': 'initial'}, layout=Layout(width='50%', margin='0 auto 20px auto'))
    dd_question.add_class("main-question-dropdown")
    display(dd_question)
    
    global dashboard_body
    dashboard_body.layout.width = '100%'
    
    dd_question.observe(update_dashboard_layout, names='value')
    display(dashboard_body)
    update_dashboard_layout({'new': dd_question.value})

# --- Run the Dashboard ---
initialize_dashboard()